In [144]:
import numpy as np
import pandas as pd

In [231]:
#Define Entropy and information Gain
def Entropy(col):
    N=col.shape[0]
    ent=0.0
    Y=np.unique(col,return_counts=True)
    for i in Y[1]:
        p=i/N
        ent+=-(p*np.log2(p))
    return ent
        
    

In [232]:
def split_data(x_data,f_key,f_val):
    x_left=pd.DataFrame([],columns=x_data.columns)#Creates an empty pandas dataframe with given columns
    x_right=pd.DataFrame([],columns=x_data.columns)
    for j in range(x_data.shape[0]):
        val=x_data[f_key].loc[j]
        if val<f_val:
            x_left=x_left.append(x_data.loc[j])
        else:
            x_right=x_right.append(x_data.loc[j])
    return x_left,x_right
                
            

In [233]:
def Info_gain(x_data,f_key,f_val):
    left,right=split_data(x_data,f_key,f_val)
    l=float(left.shape[0]/x_data.shape[0])
    r=float(right.shape[0]/x_data.shape[0])
    sum = Entropy(X['Survived'])-(l*Entropy(left['Survived'])+r*Entropy(right['Survived']))
    return sum

In [238]:
L=[]
features=['Pclass','Sex','Age','SibSp','Parch','Fare']
for i in features:
    val=Info_gain(X,i,X[i].mean())
    L.append(val)
    print(features[np.argmax(L)])

Pclass
Sex
Sex
Sex
Sex
Sex


In [343]:
features

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']

In [313]:
#Creating a decision tree
class Decision_tree:
    def __init__(self,depth=0,max_depth=3):
        self.left=None
        self.right=None
        self.f_key=None
        self.f_val=None
        self.depth=depth
        self.max_depth=max_depth
        self.target=None
    def train(self,X_Train):
    
        features=['Pclass','Sex','Age','SibSp','Parch','Fare']
        L=[]
        for i in features:
            val=Info_gain(X_Train,i,X_Train[i].mean())
            L.append(val)
        self.f_key=features[np.argmax(L)]
        self.f_val=X_Train[self.f_key].mean()
        print('Making Features in',self.f_key)
        data_left,data_right=split_data(X_Train,self.f_key,self.f_val)
        data_left=data_left.reset_index(drop=True)
        data_right=data_right.reset_index(drop=True)
            #Base Case --->when a leaf node occurs
        if data_left.shape[0]==0 or data_right.shape[0]==0:
            if X_Train.Survived.mean()>=0.5:
                self.target=1
            else:
                self.target=0
            return
            #Base Case ---->when max_depth reached
        if(self.depth>=self.max_depth):
            if X_Train.Survived.mean() >= 0.5:
                self.target=1
            else:
                self.target=0
            return
            #Recursive case
            #left-Tree
        self.left=Decision_tree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(data_left)
            #Right-Tree
        self.right=Decision_tree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(data_right)
            #Set Target After Every Node
        if(self.depth>=self.max_depth):
            if X_Train.Survived.mean() >=0.5:
                self.target=1
            else:
                self.target=0
            return
    def Traverse_Tree(self,test):
        if test[self.f_key]>self.f_val:
                #Go to right
            
                #else if right side is not present we will return target
            if self.right is None:
                return self.target
                
            return self.right.Traverse_Tree(test)
        else:
            if self.left is None:
                return self.target
            return self.left.Traverse_Tree(test)

In [323]:
split=0.7
x_train_set=X[:int(split*X.shape[0])]
x_test_set=X[int(split*X.shape[0]):]
x_test_set=x_test_set.reset_index(drop=True)

y_actual=x_test_set['Survived']

In [316]:
x_test_set=x_test_set.drop(columns='Survived')

In [317]:
dt=Decision_tree()
dt.train(x_train_set)

Making Features in Sex
Making Features in Pclass
Making Features in Age
Making Features in SibSp
Making Features in SibSp
Making Features in Parch
Making Features in SibSp
Making Features in Fare
Making Features in Fare
Making Features in Parch
Making Features in Fare
Making Features in Age
Making Features in Pclass
Making Features in Age
Making Features in SibSp


In [318]:
y_pred=[]
for ix in range(x_test_set.shape[0]):
    val=dt.Traverse_Tree(x_test_set.loc[ix])
    y_pred.append(val)

In [324]:
np.sum(y_actual==y_pred)/y_actual.shape[0]

0.8134328358208955

In [326]:
#Now I will load the data for titanic challenge

In [327]:
titanic=pd.read_csv('./titanic.csv')

In [329]:
titanic.head(5)#Now we will remove some not useful columns from dataset

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [330]:
not_useful=['PassengerId','Name','Ticket','Cabin','Embarked']

In [337]:
clean_data=titanic.drop(columns=not_useful)

In [341]:
clean_data=clean_data.fillna(clean_data['Age'].mean())#Filling theNAN values with mean age

In [342]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int64(4), object(1)
memory usage: 48.8+ KB
